In [ ]:
#!/usr/bin/env python3
'''
Main script
'''

import os
from pymatgen.io.vasp.inputs import Poscar
from pymatgen.core import Structure

from CrySPY.interface import select_code
from CrySPY.job.ctrl_job import Ctrl_job
from CrySPY.IO import read_input as rin
from CrySPY.start import cryspy_init, cryspy_restart

from aiida_lammps.tests.utils import (
    get_or_create_local_computer, get_or_create_code)
from aiida_lammps.tests.utils import lammps_version

from aiida.engine import run_get_node, run, submit
from aiida.plugins import DataFactory, WorkflowFactory
import numpy as np
from aiida.orm import Code
from aiida.orm import Str, Dict, Int, List, Float
from aiida.engine import calcfunction, WorkChain, append_
from aiida.plugins import DataFactory
from itertools import cycle
import os
import io
from configparser import ConfigParser

from CrySPY.gen_struc.random.gen_pyxtal import Rnd_struc_gen_pyxtal


In [ ]:
import aiida
aiida.load_profile()


In [ ]:
SIMULATOR_PREFIX = 'simulator_'
ID_PREFIX = 'ID_'


In [ ]:
!pwd

In [ ]:
import os
import numpy as np


from aiida.plugins import DataFactory
from aiida.orm import Code
from aiida.orm import Str, Dict, List, Int
from aiida.engine import calcfunction, WorkChain

from CrySPY.IO import read_input as rin
from pymatgen.io.vasp.inputs import Poscar
from pymatgen.core import Structure

# load types
StructureData = DataFactory('structure')
FolderData = DataFactory('folder')
SinglefileData = DataFactory('singlefile')
ArrayData = DataFactory('array')
LammpsPotential = DataFactory('lammps.potential')
TrajectoryData = DataFactory('array.trajectory')

# PandasFrameData = DataFactory('cryspy.dataframe')
PandasFrameData = DataFactory('dataframe.frame')

ConfigparserData = DataFactory('cryspy.configparser')
StructurecollectionData = DataFactory('cryspy.structurecollection')
EAData = DataFactory('cryspy.ea_data')
EAidData = DataFactory('cryspy.ea_id_data')
RSidData = DataFactory('cryspy.rs_id_data')


In [ ]:
pwd = os.getcwd()
cryspy_in = Str(os.path.join(pwd, 'cryspy.in0'))
inputs = {'cryspy_in': cryspy_in}
initialize_WorkChain = WorkflowFactory('cryspy.initial_structures')
initial_result = run(initialize_WorkChain, **inputs)


In [ ]:
initial_result


# output data

- initial_structures: initial structures generated randomly.
- rslt_data: empty dataframe 
- id_data: data for job queueing
- stat: configparser data
- cryspy_in: rin data (Many are duplicated to the content in stat.)

cryspy_in will be deleted in future.

In [ ]:
initial_structures_node = initial_result["initial_structures"]
rslt_data_node = initial_result["rslt_data"]
id_data_node = initial_result["id_data"]
cryspy_in_node = initial_result["cryspy_in"]

In [ ]:
# This package will read cryspy.in. But it is copied. 
import shutil
shutil.copy("cryspy.in0", "cryspy.in") 


# 

# select structures to optimize

It selects all in RS.
It selects some of them in the other algorithms.


In [ ]:
inputs = {"id_data":  id_data_node, "initial_structures":initial_structures_node }

select_structure_to_run_WorkChain = WorkflowFactory('cryspy.select_structures')
select_structure_result = run(select_structure_to_run_WorkChain, **inputs)
select_structure_result

In [ ]:
work_path_node = select_structure_result["work_path"]
structures_to_optimize_node = select_structure_result["selected_structures"]
id_node = select_structure_result["id_data"]


queues and running jobs are controlled by AiiDA.
Thus, the queues in id_nodes are cleared. 

The structures in structures_to_optimize_node
will be optimized in work_path_node.

keys in dict can be integer, but Dict in AiiDA doesn't accept integer key.
Thus, the keys in work_path_node is string.

In [ ]:
print("structures to opimize")
print(structures_to_optimize_node.structurecollection.keys())
print(work_path_node.get_dict().keys())

In [ ]:
# id_node is cleared.
print("id_node")
print(id_node.rs_id_data)

In [ ]:
# define or load lammps.optimize code.

computer_local = 'localhost'
code_lammps_opt = get_or_create_code('lammps.optimize',
                                     computer_local, 'lammps')
meta_options = {
    "resources": {
        "num_machines": 1,
        "num_mpiprocs_per_machine": 4}
}
code_lammps_opt.label


In [ ]:
# define lammps potentials
pair_style = 'tersoff'
potential_dict = {
    'Ga Ga Ga': '1.0 0.007874 1.846 1.918000 0.75000 -0.301300 1.0 1.0 1.44970 410.132 2.87 0.15 1.60916 535.199',
    'N  N  N': '1.0 0.766120 0.000 0.178493 0.20172 -0.045238 1.0 1.0 2.38426 423.769 2.20 0.20 3.55779 1044.77',
    'Ga Ga N': '1.0 0.001632 0.000 65.20700 2.82100 -0.518000 1.0 0.0 0.00000 0.00000 2.90 0.20 0.00000 0.00000',
    'Ga N  N': '1.0 0.001632 0.000 65.20700 2.82100 -0.518000 1.0 1.0 2.63906 3864.27 2.90 0.20 2.93516 6136.44',
    'N  Ga Ga': '1.0 0.001632 0.000 65.20700 2.82100 -0.518000 1.0 1.0 2.63906 3864.27 2.90 0.20 2.93516 6136.44',
    'N  Ga N ': '1.0 0.766120 0.000 0.178493 0.20172 -0.045238 1.0 0.0 0.00000 0.00000 2.20 0.20 0.00000 0.00000',
    'N  N  Ga': '1.0 0.001632 0.000 65.20700 2.82100 -0.518000 1.0 0.0 0.00000 0.00000 2.90 0.20 0.00000 0.00000',
    'Ga N  Ga': '1.0 0.007874 1.846 1.918000 0.75000 -0.301300 1.0 0.0 0.00000 0.00000 2.87 0.15 0.00000 0.00000'}
potential = DataFactory("lammps.potential")(
    type=pair_style, data=potential_dict
)
potential.attributes


In [ ]:
# lammps.optimize parameters
parameters = DataFactory('dict')(dict={
    'lammps_version': lammps_version(),
    'output_variables': ["temp", "etotal", "pe", "ke" , 
                         "pxx", "pyy", "pzz", "pxy", "pxz", "pyz"],
    'thermo_keywords': ["pxx", "pyy", "pzz", "pxy", "pxz", "pyz"],
    'units': 'metal',
    'relax': {
        'type': 'aniso',
        'pressure': 0.0,
        'vmax': 0.005,
    },
    "minimize": {
        'style': 'cg',
        'energy_tolerance': 1.0e-5,
        'force_tolerance': 1.0e-3,
        'max_iterations': 1000,
        'max_evaluations': 10000
        }
})


All the structures will be optimized.

In [ ]:
if len(structures_to_optimize_node.structurecollection.keys())>0 and len(work_path_node.get_dict())>0:
    inputs = {'code_string': Str(code_lammps_opt.label),
             "initial_structures": structures_to_optimize_node, 
             "cwd": work_path_node,
             "potential": potential,
             "parameters": parameters}
    optimization_simulator_lammps_WorkChain = WorkflowFactory('cryspy.optimize_structures_lammps')
    optimize_result, node = run_get_node(optimization_simulator_lammps_WorkChain, **inputs)
    

In [ ]:
optimize_result


# outputs

In [ ]:
# index and total energies.
print(optimize_result["results"].get_dict())


In [ ]:
# final_structures
# Its format is {0: pymatgen.core.Structure, 1: pymatgen.core.Structure, ...}
all_optimized_structures_node = optimize_result["final_structures"]
print(all_optimized_structures_node.structurecollection.keys())
print("\n0th structure\n")
print(all_optimized_structures_node.structurecollection[0])


# summary

In [ ]:
# optの対称性の登録
from aiida_cryspy.common import generate_rlst

rslt_data_node_new = generate_rlst(initial_structures_node, all_optimized_structures_node, 
                              optimize_result["results"], 
                              id_node, rslt_data_node, cryspy_in_node)

# rslt for new EA
rslt_data_node_new.df


In [ ]:
print("done all for RS")
